In [1]:
!pip3 install torch torchaudio torchvision torchtext torchdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 31.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
# Libraries

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
# Importing the dataset
movies = pd.read_csv("/content/drive/MyDrive/Boltzmann's Machine/Boltzmann_Machines/ml-1m/movies.dat",sep = '::' , header = None , engine = 'python',encoding = 'latin-1')

In [4]:
users = pd.read_csv("/content/drive/MyDrive/Boltzmann's Machine/Boltzmann_Machines/ml-1m/users.dat",sep = '::' , header = None , engine = 'python',encoding = 'latin-1')
ratings = pd.read_csv("/content/drive/MyDrive/Boltzmann's Machine/Boltzmann_Machines/ml-1m/ratings.dat",sep = '::' , header = None , engine = 'python',encoding = 'latin-1')

In [5]:

training_Set = pd.read_csv("/content/drive/MyDrive/Boltzmann's Machine/Boltzmann_Machines/ml-100k/u1.base" , delimiter = '\t')

In [6]:
training_Set = np.array(training_Set , dtype = 'int')

In [7]:
# Similar processing for the test dataset

test_set = pd.read_csv("/content/drive/MyDrive/Boltzmann's Machine/Boltzmann_Machines/ml-100k/u1.test" , delimiter = '\t')
test_set = np.array(test_set , dtype = 'int')

In [8]:
# total no of users
nb_users = int(max(max(training_Set[: , 0]),max(test_set[: , 0])))
print(nb_users)

943


In [9]:
# total no of movies
nb_movies = int(max(max(training_Set[: , 1]),max(test_set[: , 1])))
print(nb_movies)

1682


In [10]:
# creating a matrix for train_set and test_set where rows is users and columns is movies and cells are ratings
# as for the neural n/w parameter where observation as row and features as column

def convert(data):
  new_data = []
  for id_user in range(1 , nb_users + 1):
    id_movie = data[: , 1][data[: , 0] == id_user]
    id_rating = data[: , 2][data[: , 0] == id_user]
    ratings = np.zeros(nb_movies)
    ratings[id_movie - 1] = id_rating
    new_data.append(list(ratings))
  return new_data



In [11]:
train_dataset = convert(training_Set)
test_dataset = convert(test_set)

In [12]:
len(train_dataset) , len(test_dataset) # No of users

(943, 943)

In [13]:
len(train_dataset[0]) , len(test_dataset[0]) # No of movies

(1682, 1682)

In [14]:
# As this structure is usually created for deep learning network where lines are the observation for the n/w and
# column are the features which is the input node of the n/w
train_dataset  = torch.FloatTensor(train_dataset)
test_dataset = torch.FloatTensor(test_dataset)

In [15]:
# Now to convert the ratings into binary rating 0 and 1 as an input of the REstricted Boltzmann Machine
# where 0  is not liked and 1 is liked

train_dataset[train_dataset == 0] = -1
# user that does not watch that movie

In [16]:
train_dataset[train_dataset == 1] = 0
train_dataset[train_dataset == 2] = 0
train_dataset[train_dataset >= 3] = 1

In [17]:
test_dataset[test_dataset == 0] = -1 # user that does not watch that movie

In [18]:
test_dataset[test_dataset == 1] = 0
test_dataset[test_dataset == 2] = 0
test_dataset[test_dataset >= 3] = 1

In [60]:
# Now set up the architecture of the RBM
class RBM():
  def __init__(self , nv,nh): #nv = no of visible nodes and nh = no of hidden nodes
    self.W = torch.randn(nh,nv) # weight probabilites of visible nodes when hidden node is given
    self.a = torch.randn(1,nh) # bias probability of hidden node and batch_size = 1
    self.b = torch.randn(1,nv) # bias probability of visible node and batch_size = 1

  def sample_h(self , x):
    wx = torch.mm(x , self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation) # Sigmoid Activation
    return p_h_given_v , torch.bernoulli(p_h_given_v)

  def sample_v(self,y):
    wy = torch.mm(y,self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h , torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += torch.mm(ph0,v0) - torch.mm(phk , vk)
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)



In [35]:
nv = len(train_dataset[0])
print(nv)

1682


In [65]:
nv = len(train_dataset[0])  # Assuming train_dataset is properly defined
nh = 100  # Number of hidden units
batch_size = 100

rbm = RBM(nv, nh)
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = train_dataset[id_user:id_user+batch_size]
        v0 = train_dataset[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0])) # Average Distance here
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3079)
epoch: 2 loss: tensor(0.2463)
epoch: 3 loss: tensor(0.2469)
epoch: 4 loss: tensor(0.2549)
epoch: 5 loss: tensor(0.2597)
epoch: 6 loss: tensor(0.2715)
epoch: 7 loss: tensor(0.2708)
epoch: 8 loss: tensor(0.2522)
epoch: 9 loss: tensor(0.2513)
epoch: 10 loss: tensor(0.2510)


In [66]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = train_dataset[id_user:id_user+1]
    vt = test_dataset[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) # Average Distance here
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2351)
